In [59]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
from tqdm import tqdm
import utils
from dataset import StateActionEffectDataset
import environment
import copy

run_id = "m0qqg0gl"


In [61]:
run.config[]

{'lr': 0.0005,
 'coeff': 100,
 'epoch': 200,
 'device': 'cuda',
 'gumbel_t': 1,
 'state_dim': 10,
 'action_dim': 8,
 'batch_norm': True,
 'batch_size': 128,
 'effect_dim': 18,
 'hidden_dim': 128,
 'latent_dim': 8,
 'gumbel_hard': False,
 'save_folder': 'save/deneme',
 'dataset_name': 'v5',
 'n_hidden_layers': 4,
 'n_attention_heads': 4,
 'n_attention_layers': 4}

In [62]:
run = wandb.init(entity="colorslab",project="multideepsym", resume="must", id=run_id)
run = wandb.Api().run("colorslab/multideepsym/" + run_id)
model = utils.create_model_from_config(run.config)
model.load("_best", from_wandb=True)

epoch,200
lr,0.0005
train_loss,1.15007
val_loss,1.55961


RuntimeError: Error(s) in loading state_dict for Sequential:
	Missing key(s) in state_dict: "0.layers.4.batch_norm.weight", "0.layers.4.batch_norm.bias", "0.layers.4.batch_norm.running_mean", "0.layers.4.batch_norm.running_var", "0.layers.6.weight", "0.layers.6.bias", "0.layers.6.batch_norm.weight", "0.layers.6.batch_norm.bias", "0.layers.6.batch_norm.running_mean", "0.layers.6.batch_norm.running_var", "0.layers.8.weight", "0.layers.8.bias". 
	size mismatch for 0.layers.4.weight: copying a param with shape torch.Size([8, 128]) from checkpoint, the shape in current model is torch.Size([128, 128]).
	size mismatch for 0.layers.4.bias: copying a param with shape torch.Size([8]) from checkpoint, the shape in current model is torch.Size([128]).

In [2]:
dataset = StateActionEffectDataset("v5", "")

In [52]:
count = 0
for e in dataset.effect:
    if torch.any(e[:,2].abs().max() > 0.1):
        count += 1
( count)/ len(dataset) * 100

40.87878787878788

In [56]:
k=107

In [54]:
import plotly.graph_objects as go
import plotly.express as px
sample = dataset[k]
s = copy.deepcopy(sample["state"]).numpy()
s = np.concatenate( [s,np.arange(0,13,1).reshape(-1,1)], axis = 1)
extensions = []
for obj in s:
    if obj[9] > 3:
        ext = copy.deepcopy(obj)
        ext2 = copy.deepcopy(obj)
        ds = 0.125
        
        ext[0] += ds * ext[7]
        ext[1] += ds * ext[8]
        ext2[0] -= ds * ext[7]
        ext2[1] -= ds * ext[8]
        extensions.append(ext)
        extensions.append(ext2)
extensions = np.array(extensions)
s = np.concatenate([s, np.array(extensions)], axis = 0)
fig = go.Figure()
fig = px.scatter_3d(s, x = 0, y = 1, z = 2, color = 10, hover_data=[10, 9] ,labels = {"0" : "x", "1": "y", "2":"z", "9":"type", "10": "id"}, opacity = 0.7)
fig.show()
# input()

In [36]:
s = copy.deepcopy(sample["action"]).numpy()
s = np.concatenate( [np.arange(0,13,1).reshape(-1,1),s], axis = 1)
s

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 4.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 5.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 6.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 7.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 8.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 9.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [10.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [11.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [12.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [ ]:
def single_iteration_data(data):
    for key in ["action", "state", "effect"]:
        data[key] = data[key].unsqueeze(0).expand(-1, 13, -1)
    data["pad_mask"] = data["pad_mask"].unsqueeze(0).expand(-1, 13)
    return data

In [ ]:
sample = single_iteration_data(val_set.__getitem__(i))
with torch.no_grad():
    loss, pred = model.loss_with_pred(sample)
print(loss)